In [1]:
"""
Token F1 Calculator for Financial QA Evaluation
================================================

This module provides token-level F1 score calculation for evaluating
text-based answers (novel-generated and domain-relevant questions).

Features:
- Token-level precision, recall, and F1
- Text normalization (lowercase, punctuation removal, etc.)
- Optional stopword removal
- Detailed token analysis (common, missing, extra)

Author: Financial QA Evaluation System
Version: 1.0
"""

import re
import string
from typing import Dict, Any, List, Set, Optional


# Standard English stopwords (common words that don't carry much meaning)
ENGLISH_STOPWORDS = {
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from',
    'has', 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the',
    'to', 'was', 'were', 'will', 'with', 'the', 'this', 'but', 'they',
    'have', 'had', 'what', 'when', 'where', 'who', 'which', 'why', 'how'
}


def normalize_text(text: str, remove_punctuation: bool = True) -> str:
    """
    Normalize text for token comparison.
    
    Args:
        text: Input text
        remove_punctuation: Whether to remove punctuation
    
    Returns:
        Normalized text
    
    Examples:
        >>> normalize_text("The Consumer Segment!")
        'the consumer segment'
        
        >>> normalize_text("3M's revenue")
        '3ms revenue'
    """
    if not text:
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove punctuation if requested
    if remove_punctuation:
        # Keep alphanumeric and spaces, remove everything else
        text = re.sub(r'[^\w\s]', ' ', text)
    
    # Normalize whitespace
    text = ' '.join(text.split())
    
    return text


def tokenize(text: str, normalize: bool = True, remove_stopwords: bool = False) -> List[str]:
    """
    Tokenize text into words.
    
    Args:
        text: Input text
        normalize: Whether to normalize text first
        remove_stopwords: Whether to remove stopwords
    
    Returns:
        List of tokens
    
    Examples:
        >>> tokenize("The consumer segment shrunk by 0.9% organically.")
        ['the', 'consumer', 'segment', 'shrunk', 'by', '0', '9', 'organically']
        
        >>> tokenize("The consumer segment shrunk by 0.9% organically.", remove_stopwords=True)
        ['consumer', 'segment', 'shrunk', '0', '9', 'organically']
    """
    if normalize:
        text = normalize_text(text, remove_punctuation=True)
    
    # Split on whitespace
    tokens = text.split()
    
    # Remove stopwords if requested
    if remove_stopwords:
        tokens = [t for t in tokens if t not in ENGLISH_STOPWORDS]
    
    return tokens


def token_f1(
    gold_answer: str,
    generated_answer: str,
    normalize: bool = True,
    remove_stopwords: bool = False,
    return_details: bool = True
) -> Dict[str, Any]:
    """
    Calculate token-level F1 score between gold and generated answers.
    
    This metric is used for:
    - novel-generated questions (always)
    - domain-relevant questions with short/medium answers
    
    Args:
        gold_answer: The gold standard answer
        generated_answer: The generated answer to evaluate
        normalize: Whether to normalize text (lowercase, remove punctuation)
        remove_stopwords: Whether to remove common stopwords
        return_details: If True, return full details; if False, return only scores
    
    Returns:
        Dictionary containing:
            - f1: float - F1 score (0-1)
            - precision: float - Precision (0-1)
            - recall: float - Recall (0-1)
            - gold_tokens: List[str] - Tokens from gold answer
            - gen_tokens: List[str] - Tokens from generated answer
            - common_tokens: Set[str] - Tokens present in both
            - missing_tokens: Set[str] - Tokens in gold but not in generated
            - extra_tokens: Set[str] - Tokens in generated but not in gold
            - gold_token_count: int - Number of gold tokens
            - gen_token_count: int - Number of generated tokens
            - common_token_count: int - Number of common tokens
    
    Examples:
        >>> token_f1(
        ...     "The consumer segment shrunk by 0.9% organically.",
        ...     "The Consumer segment has dragged down 3M's overall growth in 2022."
        ... )
        {'f1': 0.4, 'precision': 0.25, 'recall': 1.0, ...}
        
        >>> token_f1(
        ...     "Cross currency swaps. Its notional value was $32,502 million.",
        ...     "Cross currency swaps had the highest notional value in FY 2021, at $32,502 million."
        ... )
        {'f1': 0.73, 'precision': 0.73, 'recall': 0.73, ...}
    """
    
    # Tokenize both answers
    gold_tokens = tokenize(gold_answer, normalize=normalize, remove_stopwords=remove_stopwords)
    gen_tokens = tokenize(generated_answer, normalize=normalize, remove_stopwords=remove_stopwords)
    
    # Convert to sets for comparison (but keep lists for counts)
    gold_set = set(gold_tokens)
    gen_set = set(gen_tokens)
    
    # Special case: both empty
    if not gold_set and not gen_set:
        return {
            'f1': 1.0,
            'precision': 1.0,
            'recall': 1.0,
            'gold_tokens': gold_tokens,
            'gen_tokens': gen_tokens,
            'common_tokens': set(),
            'missing_tokens': set(),
            'extra_tokens': set(),
            'gold_token_count': 0,
            'gen_token_count': 0,
            'common_token_count': 0,
            'gold_unique_count': 0,
            'gen_unique_count': 0,
        } if return_details else {'f1': 1.0, 'precision': 1.0, 'recall': 1.0}
    
    # Calculate overlaps
    common = gold_set & gen_set
    missing = gold_set - gen_set
    extra = gen_set - gold_set
    
    # Calculate metrics
    common_count = len(common)
    gold_count = len(gold_set)
    gen_count = len(gen_set)
    
    # Precision: what fraction of generated tokens are correct
    precision = common_count / gen_count if gen_count > 0 else 0.0
    
    # Recall: what fraction of gold tokens are captured
    recall = common_count / gold_count if gold_count > 0 else 0.0
    
    # F1: harmonic mean of precision and recall
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    
    result = {
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }
    
    if return_details:
        result.update({
            'gold_tokens': gold_tokens,
            'gen_tokens': gen_tokens,
            'common_tokens': common,
            'missing_tokens': missing,
            'extra_tokens': extra,
            'gold_token_count': len(gold_tokens),
            'gen_token_count': len(gen_tokens),
            'common_token_count': common_count,
            'gold_unique_count': gold_count,
            'gen_unique_count': gen_count,
        })
    
    return result


def batch_token_f1(
    gold_answers: List[str],
    generated_answers: List[str],
    normalize: bool = True,
    remove_stopwords: bool = False
) -> Dict[str, Any]:
    """
    Calculate token F1 for multiple answer pairs.
    
    Args:
        gold_answers: List of gold standard answers
        generated_answers: List of generated answers (same length)
        normalize: Whether to normalize text
        remove_stopwords: Whether to remove stopwords
    
    Returns:
        Dictionary containing:
            - results: List of individual results
            - mean_f1: Mean F1 score
            - mean_precision: Mean precision
            - mean_recall: Mean recall
            - median_f1: Median F1 score
            - scores: List of F1 scores for distribution analysis
    
    Example:
        >>> gold = [
        ...     "The consumer segment shrunk by 0.9% organically.",
        ...     "Cross currency swaps. Its notional value was $32,502 million."
        ... ]
        >>> generated = [
        ...     "The Consumer segment.",
        ...     "Cross currency swaps had the highest notional value at $32,502 million."
        ... ]
        >>> results = batch_token_f1(gold, generated)
        >>> print(f"Mean F1: {results['mean_f1']:.2f}")
    """
    
    if len(gold_answers) != len(generated_answers):
        raise ValueError(
            f"Length mismatch: {len(gold_answers)} gold vs {len(generated_answers)} generated"
        )
    
    results = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    
    for gold, gen in zip(gold_answers, generated_answers):
        result = token_f1(
            gold, gen, 
            normalize=normalize, 
            remove_stopwords=remove_stopwords,
            return_details=True
        )
        results.append(result)
        f1_scores.append(result['f1'])
        precision_scores.append(result['precision'])
        recall_scores.append(result['recall'])
    
    # Calculate statistics
    mean_f1 = sum(f1_scores) / len(f1_scores) if f1_scores else 0.0
    mean_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0.0
    mean_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0.0
    
    # Median
    sorted_f1 = sorted(f1_scores)
    median_f1 = sorted_f1[len(sorted_f1)//2] if sorted_f1 else 0.0
    
    return {
        'results': results,
        'total': len(gold_answers),
        'mean_f1': mean_f1,
        'mean_precision': mean_precision,
        'mean_recall': mean_recall,
        'median_f1': median_f1,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
    }


def token_overlap_ratio(
    gold_answer: str,
    generated_answer: str,
    normalize: bool = True
) -> float:
    """
    Simple token overlap ratio (Jaccard similarity).
    Alternative simpler metric to F1.
    
    Args:
        gold_answer: Gold standard answer
        generated_answer: Generated answer
        normalize: Whether to normalize text
    
    Returns:
        Overlap ratio (0-1): |intersection| / |union|
    
    Example:
        >>> token_overlap_ratio("consumer segment", "consumer segment growth")
        0.67  # 2 common tokens out of 3 total unique tokens
    """
    gold_tokens = set(tokenize(gold_answer, normalize=normalize, remove_stopwords=False))
    gen_tokens = set(tokenize(generated_answer, normalize=normalize, remove_stopwords=False))
    
    if not gold_tokens and not gen_tokens:
        return 1.0  # Both empty
    
    intersection = gold_tokens & gen_tokens
    union = gold_tokens | gen_tokens
    
    return len(intersection) / len(union) if union else 0.0


def _test_token_f1():
    """Quick sanity tests for token_f1"""
    
    print("Running quick tests for token_f1()...")
    
    test_cases = [
        # (gold, generated, expected_f1_range, description)
        (
            "The consumer segment shrunk by 0.9% organically.",
            "The Consumer segment has dragged down 3M's overall growth in 2022.",
            (0.2, 0.5),
            "Novel-generated: partial overlap"
        ),
        (
            "Cross currency swaps.",
            "Cross currency swaps had the highest notional value.",
            (0.4, 0.7),
            "Novel-generated: good overlap"
        ),
        (
            "Yes. It decreased.",
            "Yes. It decreased.",
            (0.95, 1.0),
            "Exact match"
        ),
        (
            "The quick ratio is 1.57",
            "The quick ratio is approximately 1.57",
            (0.8, 1.0),
            "Domain-relevant: close match with extra word"
        ),
        (
            "AES has converted inventory 9.5 times in FY 2022.",
            "AES Corporation sold its inventory roughly 12 times in FY2022.",
            (0.3, 0.6),
            "Domain-relevant: some overlap, different numbers"
        ),
    ]
    
    passed = 0
    failed = 0
    
    for gold, gen, (min_f1, max_f1), desc in test_cases:
        result = token_f1(gold, gen)
        f1 = result['f1']
        
        if min_f1 <= f1 <= max_f1:
            passed += 1
            print(f"✓ {desc:50} F1={f1:.3f} P={result['precision']:.3f} R={result['recall']:.3f}")
        else:
            failed += 1
            print(f"✗ {desc:50}")
            print(f"  Expected F1 in [{min_f1}, {max_f1}], got {f1:.3f}")
            print(f"  Common tokens: {result['common_tokens']}")
            print(f"  Missing: {result['missing_tokens']}")
    
    print(f"\nResults: {passed} passed, {failed} failed out of {len(test_cases)} tests")
    return failed == 0


if __name__ == "__main__":
    success = _test_token_f1()
    
    if success:
        print("\n✅ All quick tests passed!")
    else:
        print("\n⚠️  Some tests failed")

Running quick tests for token_f1()...
✓ Novel-generated: partial overlap                   F1=0.300 P=0.250 R=0.375
✓ Novel-generated: good overlap                      F1=0.545 P=0.375 R=1.000
✓ Exact match                                        F1=1.000 P=1.000 R=1.000
✓ Domain-relevant: close match with extra word       F1=0.923 P=0.857 R=1.000
✓ Domain-relevant: some overlap, different numbers   F1=0.400 P=0.400 R=0.400

Results: 5 passed, 0 failed out of 5 tests

✅ All quick tests passed!


In [2]:
"""
Comprehensive Test Suite for token_f1()
========================================

Tests cover:
- Novel-generated question examples
- Domain-relevant question examples
- Normalization behavior
- Stopword removal
- Edge cases
- Real FinanceBench examples
"""


def test_basic_token_f1():
    """Test basic F1 calculation"""
    print("\n" + "="*70)
    print("TEST SUITE 1: Basic Token F1")
    print("="*70)
    
    tests = [
        # (gold, generated, description, expected_f1_min, expected_f1_max)
        ("cat dog bird", "cat dog", "Partial match - 2/3 overlap", 0.7, 0.9),
        ("cat dog", "cat dog", "Exact match", 0.99, 1.0),
        ("cat dog", "bird fish", "No overlap", 0.0, 0.0),
        ("the cat", "the dog", "50% overlap", 0.4, 0.7),
        ("abc", "abc abc abc", "Repeated tokens collapse to unique", 0.99, 1.0),  # Sets collapse duplicates
    ]
    
    passed, failed = 0, 0
    for gold, gen, desc, min_f1, max_f1 in tests:
        result = token_f1(gold, gen)
        f1 = result['f1']
        
        if min_f1 <= f1 <= max_f1:
            passed += 1
            print(f"✓ {desc:50} F1={f1:.3f}")
        else:
            failed += 1
            print(f"✗ {desc:50}")
            print(f"  Expected F1 in [{min_f1:.3f}, {max_f1:.3f}], got {f1:.3f}")
            print(f"  P={result['precision']:.3f}, R={result['recall']:.3f}")
    
    print(f"\nBasic Token F1: {passed}/{len(tests)} passed")
    return passed, failed


def test_normalization():
    """Test text normalization"""
    print("\n" + "="*70)
    print("TEST SUITE 2: Text Normalization")
    print("="*70)
    
    tests = [
        # (gold, generated, description, should_match)
        ("Consumer Segment", "consumer segment", "Case normalization", True),
        ("revenue growth", "revenue growth", "Simple match", True),
        ("shrunk by 0.9%", "shrunk by 0 9", "Punctuation removal", True),
        ("Yes.", "Yes", "Period removal", True),
        ("32502 million", "32502 million", "Same numbers", True),  # Already normalized
    ]
    
    passed, failed = 0, 0
    for gold, gen, desc, should_match in tests:
        result = token_f1(gold, gen, normalize=True)
        f1 = result['f1']
        
        # Should match means F1 should be high (>0.9)
        matches = f1 > 0.9
        
        if matches == should_match:
            passed += 1
            print(f"✓ {desc:50} F1={f1:.3f}")
        else:
            failed += 1
            print(f"✗ {desc:50}")
            print(f"  Expected match={should_match}, F1={f1:.3f}")
            print(f"  Common: {result['common_tokens']}")
    
    print(f"\nNormalization: {passed}/{len(tests)} passed")
    return passed, failed


def test_stopword_removal():
    """Test stopword removal functionality"""
    print("\n" + "="*70)
    print("TEST SUITE 3: Stopword Removal")
    print("="*70)
    
    # Test with and without stopwords
    gold = "The consumer segment shrunk by the margin"
    gen = "consumer segment shrunk margin"
    
    result_with_stopwords = token_f1(gold, gen, remove_stopwords=False)
    result_without_stopwords = token_f1(gold, gen, remove_stopwords=True)
    
    print(f"Gold: '{gold}'")
    print(f"Generated: '{gen}'")
    print(f"\nWith stopwords:")
    print(f"  F1: {result_with_stopwords['f1']:.3f}")
    print(f"  Common tokens: {result_with_stopwords['common_tokens']}")
    print(f"\nWithout stopwords:")
    print(f"  F1: {result_without_stopwords['f1']:.3f}")
    print(f"  Common tokens: {result_without_stopwords['common_tokens']}")
    
    # Without stopwords should have higher F1 since we ignore 'the', 'by'
    if result_without_stopwords['f1'] >= result_with_stopwords['f1']:
        print(f"\n✓ Stopword removal increases F1 as expected")
        return 1, 0
    else:
        print(f"\n✗ Stopword removal did not increase F1")
        return 0, 1


def test_novel_generated_examples():
    """Test real novel-generated question examples"""
    print("\n" + "="*70)
    print("TEST SUITE 4: Novel-Generated Examples from FinanceBench")
    print("="*70)
    
    tests = [
        # Example 1
        (
            "The consumer segment shrunk by 0.9% organically.",
            "The Consumer segment.",
            "Example 1a: closedbook - partial answer"
        ),
        (
            "The consumer segment shrunk by 0.9% organically.",
            "The Consumer segment has dragged down 3M's overall growth in 2022.",
            "Example 1b: oracle - expanded answer"
        ),
        
        # Example 2
        (
            "Cross currency swaps. Its notional value was $32,502 million.",
            "Interest rate swaps had the highest notional value among Verizon's derivative instruments in FY 2021.",
            "Example 2a: closedbook - wrong answer"
        ),
        (
            "Cross currency swaps. Its notional value was $32,502 million.",
            "Cross currency swaps had the highest notional value in FY 2021, at $32,502 million.",
            "Example 2b: oracle - correct with context"
        ),
        
        # Example 3
        (
            "The estimated pension benefits were $1097 million, and the estimated health care and life insurance benefits were $862 million.",
            "Approximately $1.5 billion.",
            "Example 3a: closedbook - approximate answer"
        ),
        (
            "The estimated pension benefits were $1097 million, and the estimated health care and life insurance benefits were $862 million.",
            "$1,959 million",
            "Example 3b: oracle - total only"
        ),
    ]
    
    passed, failed = 0, 0
    for gold, gen, desc in tests:
        result = token_f1(gold, gen)
        
        # Just report the scores - we're not checking specific ranges
        print(f"\n{desc}")
        print(f"  F1={result['f1']:.3f}, P={result['precision']:.3f}, R={result['recall']:.3f}")
        print(f"  Common tokens: {len(result['common_tokens'])}/{result['gold_unique_count']}")
        
        # Basic sanity check: F1 should be between 0 and 1
        if 0 <= result['f1'] <= 1:
            passed += 1
        else:
            failed += 1
            print(f"  ✗ Invalid F1 score: {result['f1']}")
    
    print(f"\nNovel-Generated Examples: {passed}/{len(tests)} passed")
    return passed, failed


def test_domain_relevant_examples():
    """Test real domain-relevant question examples"""
    print("\n" + "="*70)
    print("TEST SUITE 5: Domain-Relevant Examples from FinanceBench")
    print("="*70)
    
    tests = [
        # Example 1
        (
            "AES has converted inventory 9.5 times in FY 2022.",
            "The inventory turnover ratio for AES Corporation in FY2022 cannot be calculated without specific COGS and average inventory figures.",
            "Example 1a: closedbook - refusal"
        ),
        (
            "AES has converted inventory 9.5 times in FY 2022.",
            "AES Corporation sold its inventory roughly 12 times in FY2022; however, conventional inventory management may not be meaningful due to the nature of its business in the energy sector.",
            "Example 1b: oracle - wrong number but relevant"
        ),
        
        # Example 2
        (
            "The quick ratio has slightly improved from 0.67 times to 0.69 times between FY 2023 and FY 2022.(3.4% jump)",
            "The quick ratio's improvement or decline cannot be determined without specific financial data for FY2022 and FY2023.",
            "Example 2a: closedbook - refusal"
        ),
        (
            "The quick ratio has slightly improved from 0.67 times to 0.69 times between FY 2023 and FY 2022.(3.4% jump)",
            "AMCOR's quick ratio has improved from FY2022 to FY2023.",
            "Example 2b: oracle - simplified correct"
        ),
        
        # Example 3
        (
            "Yes. The quick ratio is 1.57, calculated as (cash and cash equivalents+Short term investments+Accounts receivable, net+receivables from related parties)/ (current liabilities).",
            "Yes, AMD has a reasonably healthy liquidity profile based on its quick ratio for FY22.",
            "Example 3a: closedbook - yes but no details"
        ),
        (
            "Yes. The quick ratio is 1.57, calculated as (cash and cash equivalents+Short term investments+Accounts receivable, net+receivables from related parties)/ (current liabilities).",
            "Yes, AMD has a reasonably healthy liquidity profile based on its quick ratio of approximately 1.57 for FY22.",
            "Example 3b: oracle - yes with ratio"
        ),
    ]
    
    passed, failed = 0, 0
    for gold, gen, desc in tests:
        result = token_f1(gold, gen)
        
        print(f"\n{desc}")
        print(f"  F1={result['f1']:.3f}, P={result['precision']:.3f}, R={result['recall']:.3f}")
        print(f"  Common tokens: {len(result['common_tokens'])}/{result['gold_unique_count']}")
        
        # Basic sanity check
        if 0 <= result['f1'] <= 1:
            passed += 1
        else:
            failed += 1
            print(f"  ✗ Invalid F1 score: {result['f1']}")
    
    print(f"\nDomain-Relevant Examples: {passed}/{len(tests)} passed")
    return passed, failed


def test_precision_recall_trade_off():
    """Test understanding of precision vs recall"""
    print("\n" + "="*70)
    print("TEST SUITE 6: Precision vs Recall Trade-off")
    print("="*70)
    
    gold = "consumer segment growth revenue"
    
    tests = [
        ("consumer segment growth revenue", "High recall, high precision"),
        ("consumer segment", "High precision, low recall"),
        ("consumer segment growth revenue extra words here", "Low precision, high recall"),
        ("completely different words", "Low precision, low recall"),
    ]
    
    passed, failed = 0, 0
    for gen, desc in tests:
        result = token_f1(gold, gen)
        
        print(f"\n{desc}")
        print(f"  Generated: '{gen}'")
        print(f"  F1={result['f1']:.3f}, P={result['precision']:.3f}, R={result['recall']:.3f}")
        
        # Just validate that metrics are in valid range
        if 0 <= result['precision'] <= 1 and 0 <= result['recall'] <= 1:
            passed += 1
        else:
            failed += 1
            print(f"  ✗ Invalid metrics")
    
    print(f"\nPrecision vs Recall: {passed}/{len(tests)} passed")
    return passed, failed


def test_edge_cases():
    """Test edge cases"""
    print("\n" + "="*70)
    print("TEST SUITE 7: Edge Cases")
    print("="*70)
    
    tests = [
        ("", "", "Both empty", 1.0),
        ("test", "", "Generated empty", 0.0),
        ("", "test", "Gold empty", 0.0),
        ("a", "a a a a", "Repeated tokens", None),  # Just check valid range
        ("yes", "Yes", "Simple case match", 1.0),
        ("no", "No.", "Simple case with punctuation", 1.0),
    ]
    
    passed, failed = 0, 0
    for gold, gen, desc, expected_f1 in tests:
        result = token_f1(gold, gen)
        f1 = result['f1']
        
        if expected_f1 is not None:
            # Check exact or approximate match
            if abs(f1 - expected_f1) < 0.01:
                passed += 1
                print(f"✓ {desc:40} F1={f1:.3f}")
            else:
                failed += 1
                print(f"✗ {desc:40}")
                print(f"  Expected F1={expected_f1:.3f}, got {f1:.3f}")
        else:
            # Just check valid range
            if 0 <= f1 <= 1:
                passed += 1
                print(f"✓ {desc:40} F1={f1:.3f}")
            else:
                failed += 1
                print(f"✗ {desc:40} F1={f1:.3f} out of range")
    
    print(f"\nEdge Cases: {passed}/{len(tests)} passed")
    return passed, failed


def test_batch_evaluation():
    """Test batch evaluation"""
    print("\n" + "="*70)
    print("TEST SUITE 8: Batch Evaluation")
    print("="*70)
    
    gold_answers = [
        "The consumer segment shrunk by 0.9% organically.",
        "Cross currency swaps. Its notional value was $32,502 million.",
        "Yes. It decreased.",
    ]
    
    generated_answers = [
        "The Consumer segment.",
        "Cross currency swaps had the highest notional value at $32,502 million.",
        "Yes. It decreased.",
    ]
    
    result = batch_token_f1(gold_answers, generated_answers)
    
    print(f"Total: {result['total']}")
    print(f"Mean F1: {result['mean_f1']:.3f}")
    print(f"Mean Precision: {result['mean_precision']:.3f}")
    print(f"Mean Recall: {result['mean_recall']:.3f}")
    print(f"Median F1: {result['median_f1']:.3f}")
    print(f"\nIndividual F1 scores: {[f'{s:.3f}' for s in result['f1_scores']]}")
    
    # Validate
    if (0 <= result['mean_f1'] <= 1 and 
        len(result['results']) == 3 and
        len(result['f1_scores']) == 3):
        print(f"\n✓ Batch evaluation working correctly")
        return 1, 0
    else:
        print(f"\n✗ Batch evaluation failed validation")
        return 0, 1


def test_token_overlap_ratio():
    """Test simpler overlap ratio metric"""
    print("\n" + "="*70)
    print("TEST SUITE 9: Token Overlap Ratio (Jaccard)")
    print("="*70)
    
    tests = [
        ("consumer segment", "consumer segment growth", "Partial overlap"),
        ("consumer segment", "consumer segment", "Exact match"),
        ("consumer segment", "growth revenue", "No overlap"),
    ]
    
    passed, failed = 0, 0
    for gold, gen, desc in tests:
        ratio = token_overlap_ratio(gold, gen)
        
        print(f"{desc:40} Overlap={ratio:.3f}")
        
        if 0 <= ratio <= 1:
            passed += 1
        else:
            failed += 1
            print(f"  ✗ Invalid ratio: {ratio}")
    
    print(f"\nToken Overlap Ratio: {passed}/{len(tests)} passed")
    return passed, failed


def run_all_tests():
    """Run all test suites"""
    print("\n" + "="*70)
    print("COMPREHENSIVE TEST SUITE FOR token_f1()")
    print("="*70)
    
    total_passed = 0
    total_failed = 0
    
    test_suites = [
        test_basic_token_f1,
        test_normalization,
        test_stopword_removal,
        test_novel_generated_examples,
        test_domain_relevant_examples,
        test_precision_recall_trade_off,
        test_edge_cases,
        test_batch_evaluation,
        test_token_overlap_ratio,
    ]
    
    for test_func in test_suites:
        passed, failed = test_func()
        total_passed += passed
        total_failed += failed
    
    print("\n" + "="*70)
    print("FINAL SUMMARY")
    print("="*70)
    print(f"Total Tests: {total_passed + total_failed}")
    print(f"✓ Passed: {total_passed}")
    print(f"✗ Failed: {total_failed}")
    print(f"Success Rate: {100 * total_passed / (total_passed + total_failed):.1f}%")
    print("="*70)
    
    return total_failed == 0


if __name__ == "__main__":
    success = run_all_tests()
    
    if success:
        print("\n🎉 ALL TESTS PASSED! 🎉")
    else:
        print("\n⚠️  SOME TESTS FAILED - Review output above")


COMPREHENSIVE TEST SUITE FOR token_f1()

TEST SUITE 1: Basic Token F1
✓ Partial match - 2/3 overlap                        F1=0.800
✓ Exact match                                        F1=1.000
✓ No overlap                                         F1=0.000
✓ 50% overlap                                        F1=0.500
✓ Repeated tokens collapse to unique                 F1=1.000

Basic Token F1: 5/5 passed

TEST SUITE 2: Text Normalization
✓ Case normalization                                 F1=1.000
✓ Simple match                                       F1=1.000
✓ Punctuation removal                                F1=1.000
✓ Period removal                                     F1=1.000
✓ Same numbers                                       F1=1.000

Normalization: 5/5 passed

TEST SUITE 3: Stopword Removal
Gold: 'The consumer segment shrunk by the margin'
Generated: 'consumer segment shrunk margin'

With stopwords:
  F1: 0.800
  Common tokens: {'consumer', 'shrunk', 'margin', 'segment'}

Wit

In [4]:
# Single comparison
result = token_f1(
    gold_answer="The consumer segment shrunk by 0.9% organically.",
    generated_answer="The Consumer segment has dragged down growth.",
    normalize=True,
    remove_stopwords=False
)

print(f"F1: {result['f1']:.3f}")
print(f"Precision: {result['precision']:.3f}")
print(f"Recall: {result['recall']:.3f}")
print(f"Common tokens: {result['common_tokens']}")
print(f"Missing tokens: {result['missing_tokens']}")

# Batch evaluation
results = batch_token_f1(
    gold_answers=[
        "The consumer segment shrunk by 0.9% organically.",
        "Cross currency swaps. Its notional value was $32,502 million."
    ],
    generated_answers=[
        "The Consumer segment.",
        "Cross currency swaps had the highest notional value at $32,502 million."
    ],
    normalize=True,
    remove_stopwords=False
)

print(f"Mean F1: {results['mean_f1']:.3f}")
print(f"Mean Precision: {results['mean_precision']:.3f}")
print(f"Mean Recall: {results['mean_recall']:.3f}")

F1: 0.400
Precision: 0.429
Recall: 0.375
Common tokens: {'segment', 'the', 'consumer'}
Missing tokens: {'organically', 'by', '0', 'shrunk', '9'}
Mean F1: 0.636
Mean Precision: 0.833
Mean Recall: 0.588
